# 데이터 병합

In [7]:
import pandas as pd
from glob import glob

In [8]:
data = glob('*.xlsx')
data

['5_문어체_조례.xlsx',
 '2_대화체.xlsx',
 '1_구어체(2).xlsx',
 '1_구어체(1).xlsx',
 '3_문어체_뉴스(2).xlsx',
 '3_문어체_뉴스(3).xlsx',
 '3_문어체_뉴스(1).xlsx',
 '4_문어체_한국문화.xlsx',
 '3_문어체_뉴스(4).xlsx',
 '6_문어체_지자체웹사이트.xlsx']

In [9]:
# 전체 데이터 중 문어체_조레, 문어제_지자체웝사이트 파일 제외하고 데이터 프레임 생성

df = pd.DataFrame(columns = ['원문','번역문'])

file_list = [ '2_대화체.xlsx',
 '1_구어체(2).xlsx',
 '1_구어체(1).xlsx',
 '3_문어체_뉴스(2).xlsx',
 '3_문어체_뉴스(3).xlsx',
 '3_문어체_뉴스(1).xlsx',
 '4_문어체_한국문화.xlsx',
 '3_문어체_뉴스(4).xlsx']

for data in file_list:
    temp = pd.read_excel(data)
    df = pd.concat([df,temp[['원문','번역문']]])
    

In [10]:
df.head()

,원문,번역문
0,이번 신제품 출시에 대한 시장의 반응은 어떤가요?,How is the market's reaction to the newly rele...
1,판매량이 지난번 제품보다 빠르게 늘고 있습니다.,The sales increase is faster than the previous...
2,그렇다면 공장에 연락해서 주문량을 더 늘려야겠네요.,"Then, we'll have to call the manufacturer and ..."
3,"네, 제가 연락해서 주문량을 2배로 늘리겠습니다.","Sure, I'll make a call and double the volume o..."
4,지난 회의 마지막에 논의했던 안건을 다시 볼까요?,Shall we take a look at the issues we discusse...


In [13]:
# 라이브러리 불러오기
import torch
import torch.nn as nn
import torch.optim as optim

from torchtext.datasets import TranslationDataset, Multi30k
from torchtext.data import Field, BucketIterator

import spacy
import numpy as np

import random
import math
import time

# 토크나이저 만들기

In [17]:
from torchtext import data 
from konlpy.tag import Okt

tokenizer = Okt()

In [18]:
def tokenize_kor(text):
    """
    한국어를 tokenizer해서 단어들을 리스트로 만든 후 reverse하여 반환
    """
    return [text_ for text_ in tokenizer.morphs(text)][::-1]

def tokenize_en(text):
    """
    영어를 split tokenizer해서 단어들을 리스트로 만드는 함수
    
    """
    return [text_ for text_ in text.split()]

# 필드 정의

SRC = data.Field(tokenize = tokenize_kor,
                init_token = '<sos>',
                eos_token = '<eos>')

TRG = data.Field(tokenize = tokenize_en,
                init_token = '<sos>',
                eos_token = '<eos>',
                lower = True)


# 데이터셋 만들기 (전체 데이터중 10만개만 사용)

In [19]:
df_shuffled=df.sample(frac=1).reset_index(drop=True)

In [115]:
from sklearn.model_selection import KFold
# 우선 전제 데이터중 10만개만 사용
df_ = df_shuffled[:100000]

train_df = df_[:95000]
test_df = df_[95000:]

In [116]:
print('trn size: ',len(train_df))
print('test size: ',len(test_df))

trn size:  95000
test size:  5000


In [117]:
train_df.to_csv('train_df.csv',index = False)
test_df.to_csv('test_df.csv',index = False)

In [118]:
from torchtext.data import TabularDataset

train_data, test_data =TabularDataset.splits(
     path='', train='train_df.csv',test= 'test_df.csv', format='csv',
        fields=[('원문', SRC), ('번역문', TRG)], skip_header=True)

In [119]:
# 학습 데이터와 검증데이터셋 분리
train_data, validation_data = train_data.split(split_ratio = 0.8, random_state = random.seed(323))

In [120]:
print('훈련 샘플의 개수 : {}'.format(len(train_data)))
print('검증 샘플의 개수 : {}'.format(len(validation_data)))

훈련 샘플의 개수 : 76000
검증 샘플의 개수 : 19000


# Vocab 생성

In [121]:
# 말뭉치 생성
SRC.build_vocab(train_data, min_freq = 2, max_size = 50000)
TRG.build_vocab(train_data, min_freq = 2, max_size = 50000)

# data loader

In [122]:
# data loader 생성
from torchtext.data import Iterator

# 하이퍼파라미터
batch_size = 128
lr = 0.001
EPOCHS = 20

train_iterator = Iterator(dataset = train_data, batch_size = batch_size)
valid_iterator = Iterator(dataset = validation_data, batch_size = batch_size)
test_iterator  = Iterator(dataset = test_data, batch_size = batch_size)

In [123]:
print('훈련 데이터의 미니 배치 수 : {}'.format(len(train_iterator)))
print('검증 데이터의 미니 배치 수 : {}'.format(len(valid_iterator)))

훈련 데이터의 미니 배치 수 : 594
검증 데이터의 미니 배치 수 : 149


# 모델 설계
## encoder
- 2 layer RNN
- back-bone으로 GRU 사용
- Layer 1: 독일어 토큰의 임베딩을 입력으로 받고 은닉상태 출력
- Layer 2 : Layer1의 은닉상태를 입력으로 받고 새로운 은닉상태 출력
- 각 layer마다 초기 은닉상태 h|_0 필요 (0으로 초기화 ?)
- 각 layer마다 context vector 'z'를 출력

In [124]:

class Encoder(nn.Module):
    """
    seq2seq의 encoder
    
    attribute
    ---
    input_dim : int
        input 데이터의 차원(= vocab size)
    emb_dim : int
        embedding layer의 차원
    hid_dim : int
        은닉 상태의 차원
    n_layers : int
        RNN 안의 레이어 개수 (여기선 2개)
    dropout : float
        사용할 드롭아웃의 비율 (오버피팅 방지하는 정규화 방법)
    """
    
    
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout = 0.2):
       
        super().__init__()

        self.hid_dim = hid_dim
        self.n_layers = n_layers

        self.embedding = nn.Embedding(input_dim, emb_dim)
        self.rnn = nn.GRU(emb_dim, hid_dim, n_layers, dropout = dropout)
        self.dropout = nn.Dropout(dropout)

    def forward(self, src):
        """
        
        파라미터 
        ---
        src : [src len, batch_size)]
            input data
        return
        ---
        hidden : [[n layers * n directions, batch size, hid dim]]
            encoder의 hidden state. decoder의 입력으로 사용됨
        """
        #src = [src len, batch_size)]
        embedded = self.dropout(self.embedding(src))
        #embeded = [src len, batch size, emb dim]
        outputs, hidden = self.rnn(embedded)
        # outputs = [src len, batch size, hid dim * n directions]
        # hidden = [n layers * n directions, batch size, hid dim]

        
        return hidden

## decoder
- Layer 1 : 직전 time-stamp로 부터 은닉 상태(s)와 cell state를 받고, 이들과 embedded token인 y_t를 입력으로 받아 새로운 은닉상태와 cell state를 만들어냄
- Layer 2 : Layer 2의 은닉 상태(s)와 Layer 2에서 직전 time-stamp의 은닉 상태(s)와 cell state를 입력으로 받아 새로운 은닉 상태와 cell state를 만들어냄
- Decoder Layer1의 첫 은닉상태(s)와 cell state = context vector (z) = Encoder Layer 1의 마지막 은닉상태(h)와 cell state
- Decoder RNN/LSTM의 맨 위 Layer의 은닉 상태를 Linear Layer인 f에 넘겨서 다음 토큰이 무엇일지 예측함
- 여기서는 GRU를 사용했기 때문에 cell state는 없음.

In [125]:
class Decoder(nn.Module) : 
    """
    seq2seq의 Decoder
    
    attribute
    ---
    output_dim : int
        출력 될 데이터의 차원, 타겟 데이터의 임베딩 차원        
    emb_dim  : int
        embedding layer의 차원
    hid_dim : int
        은닉 상태의 차원
    n_layers : int
        RNN 안의 레이어 개수 (여기선 2개)
    dropout : float
        사용할 드롭아웃의 비율 (오버피팅 방지하는 정규화 방법)
    """
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        self.emb_dim = emb_dim
        self.embedding = nn.Embedding(output_dim, emb_dim)
        self.rnn = nn.GRU(emb_dim, hid_dim, n_layers, dropout=dropout)
        self.fc_out = nn.Linear(hid_dim, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden):
        """

        파라미터
        ---
        input: [batch size]
            이전 단어
        hidden : [n layers * n directions, batch size, hid dim]
            이전 layer의 hidden state
            
        returns
        ---
        prediction : torch.tensor
            현재 sequence에서 생성된 출력 벡터(단어)
        hidden : [n layers, batch size, hid dim]
            decoder의 hidden state. 다음 decoder로 전달됨.
        """

        # 
        # 
        # Decoder에서 항상 n directions = 1
        # 따라서 hidden = [n layers, batch size, hid dim]
        # context = [n layers, batch size, hid dim]
        
        # input = [1, batch size]
        input = input.unsqueeze(0)
        
        # embedded = [1, batch size, emb dim]
        embedded = self.dropout(self.embedding(input))
        
        output, hidden = self.rnn(embedded, hidden)
        
        # output = [seq len, batch size, hid dim * n directions]
        # hidden = [n layers * n directions, batch size, hid dim]
 
        # Decoder에서 항상 seq len = n directions = 1 
        # 한 번에 한 토큰씩만 디코딩하므로 seq len = 1
        # 따라서 output = [1, batch size, hid dim]
        # hidden = [n layers, batch size, hid dim]
        
        # prediction = [batch size, output dim]
        prediction = self.fc_out(output.squeeze(0))
        
        return prediction, hidden
        

In [126]:
class Seq2Seq(nn.Module):
    """
    encoder와 decoder를 이용해 seq2seq 모델을 설계하는 class
    
    attribute
    ---
    encoder : 
        encoder 클래스
    decoder :
        decoder 클래스
    
    """
    def __init__(self, encoder, decoder):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        

        # Encoder와 Decoder의 hidden dim이 같아야 함 
        assert encoder.hid_dim == decoder.hid_dim, "encoder와 decoder의 hidden dim이 다름."
        assert encoder.n_layers == decoder.n_layers, "encoder와 decoder의 n_layers이 다름."

    def forward(self, src, trg ,teacher_forcing_ratio = 0.5):
        """ 
        seq2seq 모델을 통해 예측 값 생성

        파라미터
        ---
        src : [src len, batch size]
            input 데이터의 임베딩 차원
        trg : [trg len, batch size]
            target 데이터의 임베딩 차원
        teacher_forcing_ration : float
            teacher forcing의 비율
        
        returns
        ---
        outputs : [trg len, batch size, output dim]
            seq2seq를 통해 생성된 단어의 벡터 
        """

        # 
        # trg = [trg len, batch size]

        trg_len = trg.shape[0]
        batch_size = trg.shape[1]
        trg_vocab_size = self.decoder.output_dim
        
        # decoder 결과를 저장할 텐서
        outputs = torch.zeros([trg_len,batch_size , trg_vocab_size])

        # encoder의 마지막 은닉 상태가 Deocder의 초기 은닉상태로 쓰임
        hidden = self.encoder(src)

        # decoder에 들어갈 첫 input은 <sos>토큰
        input = trg[0,:]

        # target length만큼 반복
        # range(0,trg_len)이 아니라 range(1,trg_len)인 이유 : 0번째 trg는 항상 <sos>라서 그에 대한 output도 항상 0
        for t in range(1, trg_len):
            output, hidden = self.decoder(input, hidden)
            outputs[t] = output

            teacher_force = random.random() < teacher_forcing_ratio

            # 확률 가장 높게 예측한 토큰
            top1 = output.argmax(1)

            #teacher_force = 1 = true 면 trg[t]를 아니면 top1을 input으로 사용
            input = trg[t] if teacher_force else top1
        return outputs

In [127]:
input_dim = len(SRC.vocab)
output_dim = len(TRG.vocab)

# Encoder embedding dim 
enc_emb_dim = 256

# Decoder embedding dim 
dec_emb_dim = 256

hid_dim = 512
n_layers = 2

enc_dropout = 0.5
dec_dropout = 0.5

# 인코더 디코더 설정
enc = Encoder(input_dim, enc_emb_dim, hid_dim, n_layers,enc_dropout)
dec = Decoder(output_dim, dec_emb_dim, hid_dim, n_layers,dec_dropout)

device = torch.device('cuda:0')
model = Seq2Seq(enc, dec).to(device)

In [128]:
def init_weights(m):
    """
    가중치 초기화
    """
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)
model = model.apply(init_weights)

## Optimizer / Loss

In [129]:
optimizer = optim.Adam(model.parameters())

# <pad> 토큰의 index를 넘겨 받으면 오차를 계산하지 않고 ignore하기
# <pad> = padding
trg_pad_idx = TRG.vocab.stoi[TRG.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = trg_pad_idx)

## 학습 코드

In [130]:
def train(model, iterator, optimizer, criterion, clip):
    """
    모델을 학습하는 코드
    """
    model.train()
    epoch_loss=0
    
    for i, batch in enumerate(iterator):
        src = batch.원문.to(device) # [25,128]
        trg = batch.번역문.to(device) # [29,128]
        
        optimizer.zero_grad()
        
        output = model(src, trg).to(device)
        
        # trg = [trg len, batch size]
        # output = [trg len, batch size, output dim]
        output_dim = output.shape[-1]
        
        # loss 함수는 2d input으로만 계산 가능 
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        # trg = [(trg len-1) * batch size]
        # output = [(trg len-1) * batch size, output dim)]
        loss = criterion(output, trg)
        
        loss.backward()
        
        # 기울기 폭발 막기 위해 clip
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss+=loss.item()
        
    return epoch_loss/len(iterator)

In [131]:
def evaluate(model, iterator, criterion):
    """
    학습된 모델을 평가하는 코드
    """
    model.eval()
    epoch_loss = 0
    
    with torch.no_grad():
        for i, batch in enumerate(iterator):
            src = batch.원문.to(device)
            trg = batch.번역문.to(device)
            
            # teacher_forcing_ratio = 0 (아무것도 알려주면 안 됨)
            output = model(src, trg, 0).to(device)
            
            # trg = [trg len, batch size]
            # output = [trg len, batch size, output dim]
            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)
            
            # trg = [(trg len - 1) * batch size]
            # output = [(trg len - 1) * batch size, output dim]
            
            loss = criterion(output, trg)
            
            epoch_loss+=loss.item()
        
        return epoch_loss/len(iterator)

In [132]:
# function to count training time
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [133]:
import time
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 128m 33s
	Train Loss: 6.758 | Train PPL: 860.864
	 Val. Loss: 6.595 |  Val. PPL: 731.301
Epoch: 02 | Time: 130m 10s
	Train Loss: 6.043 | Train PPL: 421.068
	 Val. Loss: 6.421 |  Val. PPL: 614.908
Epoch: 03 | Time: 130m 9s
	Train Loss: 5.689 | Train PPL: 295.719
	 Val. Loss: 6.290 |  Val. PPL: 539.014
Epoch: 04 | Time: 129m 19s
	Train Loss: 5.419 | Train PPL: 225.751
	 Val. Loss: 6.167 |  Val. PPL: 476.525
Epoch: 05 | Time: 130m 35s
	Train Loss: 5.189 | Train PPL: 179.271
	 Val. Loss: 6.124 |  Val. PPL: 456.819
Epoch: 06 | Time: 131m 45s
	Train Loss: 4.988 | Train PPL: 146.671
	 Val. Loss: 6.107 |  Val. PPL: 449.047
Epoch: 07 | Time: 135m 40s
	Train Loss: 4.826 | Train PPL: 124.719
	 Val. Loss: 6.114 |  Val. PPL: 451.926
Epoch: 08 | Time: 135m 18s
	Train Loss: 4.677 | Train PPL: 107.401
	 Val. Loss: 6.132 |  Val. PPL: 460.434
Epoch: 09 | Time: 137m 29s
	Train Loss: 4.539 | Train PPL:  93.568
	 Val. Loss: 6.166 |  Val. PPL: 476.264
Epoch: 10 | Time: 133m 57s
	Train Loss

In [134]:
# 번역(translation) 함수
def translate_sentence(sentence, src_field, trg_field, model, device, max_len=50):
    """
    문장을 받아 번역 해주는 함수.
    """
    model.eval() # 평가 모드

    if isinstance(sentence, str):
        tokens = [text_ for text_ in tokenizer.morphs(sentence)][::-1]
    else:
        raise Exception("input 데이터가 str이 아닙니다.")
        

    # 처음에 <sos> 토큰, 마지막에 <eos> 토큰 붙이기
    tokens = [src_field.init_token] + tokens + [src_field.eos_token]
    print(f"전체 소스 토큰: {tokens}")

    src_indexes = [src_field.vocab.stoi[token] for token in tokens]
    print(f"소스 문장 인덱스: {src_indexes}")

    src_tensor = torch.LongTensor(src_indexes).unsqueeze(1).to(device)

    # 인코더(endocer)에 소스 문장을 넣어 문맥 벡터(context vector) 계산
    with torch.no_grad():
        hidden = model.encoder(src_tensor)

    # 처음에는 <sos> 토큰 하나만 가지고 있도록 하기
    trg_indexes = [trg_field.vocab.stoi[trg_field.init_token]]

    for i in range(max_len):
        # 이전에 출력한 단어가 현재 단어로 입력될 수 있도록
        trg_tensor = torch.LongTensor([trg_indexes[-1]]).to(device)

        with torch.no_grad():
            output, hidden = model.decoder(trg_tensor, hidden)

        pred_token = output.argmax(1).item()
        # <eos>를 만나는 순간 끝
        if pred_token == trg_field.vocab.stoi[trg_field.eos_token]:
            break
        trg_indexes.append(pred_token) # 출력 문장에 더하기

        
    # 각 출력 단어 인덱스를 실제 단어로 변환
    trg_tokens = [trg_field.vocab.itos[i] for i in trg_indexes]

    # 첫 번째 <sos>는 제외하고 출력 문장 반환
    return trg_tokens[1:]

In [138]:
sentence = '오늘 저녁은 무엇인가요?'

translate_sentence(sentence, SRC, TRG, model, device, max_len=50)

전체 소스 토큰: ['<sos>', '?', '인가요', '무엇', '은', '저녁', '오늘', '<eos>']
소스 문장 인덱스: [2, 43, 1373, 510, 12, 983, 220, 3]


['today?']

# Reference
---
- https://codlingual.tistory.com/91